In [1]:
import os
import copy
import pandas as pd
from random import shuffle
import numpy as np
import logging
import time
from tqdm import tqdm
import re
import string
import gc; gc.enable()

from nltk.corpus import stopwords                
from nltk.stem.snowball import RussianStemmer
stemmer = RussianStemmer(ignore_stopwords=False)

def clean_text(txt):
    return u" ".join([stemmer.stem(re.sub(r'\b\d+\b', '', wrd)) for wrd in str(txt).lower().strip().split(string.punctuation)
                         if wrd not in stopwords.words('russian')])

In [2]:
usecols = ['title', 'description', 'activation_date']

train = pd.read_csv('data/train.csv', parse_dates=['activation_date'], usecols=usecols)
train = train.sort_values(['activation_date']).reset_index(drop=True)

test = pd.read_csv('data/test.csv', parse_dates=['activation_date'], usecols=usecols)

In [3]:
def process_chunk(df, is_title):
    if is_title:
        df['title'].fillna('unknowntitle', inplace=True)
        df['title'] = [clean_text(title) for title in tqdm(df['title'].values)]
    else:
        df['description'].fillna('unknowndescription', inplace=True)
        df['description'] = [clean_text(desc) for desc in tqdm(df['description'].values)]
        
    return df

In [4]:
process_chunk(train, False)
process_chunk(train, True)
process_chunk(test, False)
process_chunk(test, True)

100%|████████████████████████████████████████████████████████████████████| 508438/508438 [02:58<00:00, 2843.57it/s]


,title,description,activation_date
0,отдам бесплатн,на ангарск,2017-04-18
1,продам велосипед,"продам велосипед кама ф200,в нормальном состо...",2017-04-16
2,bbk,продам новый телевизор ббк диагональ смарт т...,2017-04-17
3,вытяжка jетаир,"продам новую вытяжку в упаковке,с документами....",2017-04-17
4,коляска зима-лет,продам отличную коляску. б/у год. все вопросы...,2017-04-15
5,стекла для пластиковых окон,продаю стеклопакеты новые:/\n. 1322х652х24 /\n...,2017-04-12
6,ноутбук леново идеапад -15ибр арт.,описание: /\nноутбук леново идеапад -15ибр мож...,2017-04-17
7,ниллкин пhеном для аппле ипhон,для ипhоне (черный)/\n /\nпродаются от магази...,2017-04-18
8,флешка усб 32гб смартбу,новые/\n /\nесть синие и оранжевые/\n /\nсмарт...,2017-04-18
9,ботинк,"фирма ""кенгуру""",2017-04-12


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize

count_vectorizer_title = CountVectorizer(stop_words=stopwords.words('russian'), lowercase=True, min_df=20, max_df=.9)

title_counts = count_vectorizer_title.fit_transform(train['title'].append(test['title']))
title_counts = title_counts.log1p()


count_vectorizer_desc = TfidfVectorizer(stop_words=stopwords.words('russian'), 
                                        lowercase=True, ngram_range=(1, 2), 
                                        min_df=10, max_df=.9,
                                        smooth_idf=True, sublinear_tf =True,
                                        max_features=25000)
desc_counts = count_vectorizer_desc.fit_transform(train['description'].append(test['description']))

In [6]:
from scipy.sparse import hstack

all_text_feat_sparse = hstack([title_counts, desc_counts], format='csr')
all_text_feat_sparse.shape

(2011862, 44161)

In [7]:
import pickle
pickle.dump(all_text_feat_sparse[:len(train),:], open('train_text_sparse_new', 'wb'))
pickle.dump(all_text_feat_sparse[len(train):,:], open('test_text_sparse_new', 'wb'))

In [10]:
from sklearn.decomposition import NMF
all_text_feat_dense = NMF(n_components=150, solver='mu').fit_transform(all_text_feat_sparse)

In [11]:
pickle.dump(all_text_feat_dense[:len(train),:], open('train_text_dense_new', 'wb'))
pickle.dump(all_text_feat_dense[len(train):,:], open('test_text_dense_new', 'wb'))